In [1]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain
from constraint_solver import ConstraintSolver

In [2]:
def constraint_factory(attrs, clues):
    constrains: list[Constraint] = []
    classifier = ClueClassifier()
    for c in clues:
        clue, clue_type = classifier.classify(c)

        if clue_type == "IDENTITY":
            constrains.append(IdentityConstrain(attrs, clue))
        if clue_type == "NEXT_TO":
            constrains.append(NextToConstrain(attrs, clue))
        if clue_type == "LEFT":
            constrains.append(LeftConstrain(attrs, clue))
        if clue_type == "RIGHT":
            constrains.append(RightConstrain(attrs, clue))
        if clue_type == "DISTANCE":
            constrains.append(DistanceConstrain(attrs, clue))
        if clue_type == "DIRECT_LEFT":
            constrains.append(DirectLeftConstrain(attrs, clue))
        if clue_type == "DIRECT_RIGHT":
            constrains.append(DirectRightConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE":
            constrains.append(PositionAbsoluteConstrain(attrs, clue))
        if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
            constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
        if clue_type == "UNKNOWN":
            raise TypeError
    
    return constrains

In [3]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()

In [4]:
puzzle = gridmode.puzzle.iloc[2].lower()

attrs, clues = ppp.proccess(puzzle)

print(puzzle)

constrains: list[Constraint] = constraint_factory(attrs, clues)

Cs = ConstraintSolver(attrs, constrains)

solution = Cs.solve()

Cs.print_solution(solution)



there are 6 houses, numbered 1 to 6 from left to right, as seen from across the street. each house is occupied by a different person. each house has a unique attribute for each of the following characteristics:
 - each person has a unique name: `bob`, `alice`, `peter`, `eric`, `arnold`, `carol`
 - each mother is accompanied by their child: `fred`, `timothy`, `samantha`, `alice`, `meredith`, `bella`
 - people have unique favorite music genres: `pop`, `hip hop`, `classical`, `jazz`, `rock`, `country`
 - people have unique heights: `average`, `very tall`, `tall`, `super tall`, `very short`, `short`

## clues:
1. there is one house between the person's child is named samantha and the person who is short.
2. the person's child is named alice is bob.
3. the person who loves country music is directly left of arnold.
4. alice is the person who is tall.
5. the person who loves pop music is eric.
6. bob is somewhere to the right of the person who is super tall.
7. the person's child is named fre